# 交易时间

各品种交易时间由查询交易所网站公告信息获得。

目前，各市场各品种都采用“交易节”的形式。比如：

- 股票，2个交易节，<09:30 ~ 11:30, 13:00 ~ 15:00>
- 金融期货，2个交易节
    - 股指期货，<09:30 ~ 11:30, 13:00 ~ 15:00>
    - 国债期货，<09:30 ~ 11:30, 13:00 ~ 15:15>
- 商品期货
    - 无夜盘，3个交易节，<09:00 ~ 10:15, 10:30 ~ 11:30, 13:30 ~ 15:00>
    - 有夜盘，4个交易节：
        - 贵金属、原油：<21:00 ~ 02:30, 09:00 ~ 10:15, 10:30 ~ 11:30, 13:30 ~ 15:00>
        - 有色：<21:00 ~ 01:00, 09:00 ~ 10:15, 10:30 ~ 11:30, 13:30 ~ 15:00>
        - 其它：<21:00 ~ 23:00, 09:00 ~ 10:15, 10:30 ~ 11:30, 13:30 ~ 15:00>
        
以 SHFE.al 为例，整合为一个数据结构：

In [1]:
import datetime as dt

product_trading_time = {
    'SHFE.al': {
        'count': 4,
        'optional': 1,
        'sections': [
            dt.time(hour=21, minute=0),
            dt.time(hour=1, minute=0),
            dt.time(hour=9, minute=0),
            dt.time(hour=10, minute=15),
            dt.time(hour=10, minute=30),
            dt.time(hour=11, minute=30),
            dt.time(hour=13, minute=30),
            dt.time(hour=15, minute=0),
        ],
    },
}

print(product_trading_time)

{'SHFE.al': {'count': 4, 'optional': 1, 'sections': [datetime.time(21, 0), datetime.time(1, 0), datetime.time(9, 0), datetime.time(10, 15), datetime.time(10, 30), datetime.time(11, 30), datetime.time(13, 30), datetime.time(15, 0)]}}


沪铝的第3交易节的开盘收盘时间：

In [2]:
print(product_trading_time['SHFE.al']['sections'][(3-1)*2])
print(product_trading_time['SHFE.al']['sections'][(3-1)*2+1])

10:30:00
11:30:00


整合为一个 Python 类：

In [3]:
from typing import List

class ProductTradingTime:
    """
    品种交易时间。
    """

    def __init__(self,
                 exchange: str,
                 product: str,
                 count: int,
                 optional: int,
                 sections: List[dt.time]
                 ) -> None:

        # 交易所
        self.exchange: str = exchange.upper()

        # 品种
        self.product: str = product

        # 交易节的数量
        if count <= 0:
            raise ValueError(f'Parameter <count> should be positive integer. Got {count} instead.')
        else:
            self.count: int = count

        # 可选的交易节序号（有夜盘为1，无夜盘为0——不可选）。
        if optional < 0 or optional > 1:
            raise ValueError(f'Parameter <optional> should be 0 or 1. Got {optional} instead.')
        else:
            self.optional: int = optional

        # 交易节时间
        if len(sections) / count != 2:
            raise ValueError(f'Parameter <trading_section> should has twice number items of <count>. {product}')
        else:
            self.sections: List[dt.time] = sections

简单测试一下，还是用 SHFE.al 做例子：

In [4]:
test: ProductTradingTime = ProductTradingTime(
    exchange='SHFE',
    product='al',
    count=4,
    optional=1,
    sections=[
        dt.time(hour=21, minute=0),
        dt.time(hour=1, minute=0),
        dt.time(hour=9, minute=0),
        dt.time(hour=10, minute=15),
        dt.time(hour=10, minute=30),
        dt.time(hour=11, minute=30),
        dt.time(hour=13, minute=30),
        dt.time(hour=15, minute=0),
    ]
)

print(f'test.exchange: {test.exchange}')
print(f'test.product: {test.product}')
print(f'test.count: {test.count}')
print(f'test.optional: {test.optional}')
print(f'test.sections: {test.sections}')

test.exchange: SHFE
test.product: al
test.count: 4
test.optional: 1
test.sections: [datetime.time(21, 0), datetime.time(1, 0), datetime.time(9, 0), datetime.time(10, 15), datetime.time(10, 30), datetime.time(11, 30), datetime.time(13, 30), datetime.time(15, 0)]


改进一下：

1. 用 property 改写为只读属性；
2. 时区转换；
3. 对于无时区的时间，认定为 UTC+8 时区。

In [5]:
class ProductTradingTime:
    """
    品种交易时间。
    """
    _exchange: str  # 交易所
    _product: str   # 品种
    _count: int     # 交易节的数量
    _optional: int  # 可选的交易节序号（有夜盘为1，无夜盘为0——不可选）。
    _sections: List[dt.time]    # 交易节时间

    def __init__(self,
                 exchange: str,
                 product: str,
                 count: int,
                 optional: int,
                 sections: List[dt.time]
                 ) -> None:

        self._exchange = exchange.upper()

        self._product = product

        if count <= 0:
            raise ValueError(f'Parameter <count> should be positive integer. Got {count} instead.')
        else:
            self._count = count

        if optional < 0 or optional > 1:
            raise ValueError(f'Parameter <optional> should be 0 or 1. Got {optional} instead.')
        else:
            self._optional = optional

        if len(sections) / count != 2:
            raise ValueError(f'Parameter <trading_section> should has twice number items of <count>. {product}')
        else:
            self._sections = []
            for item in sections:
                if item.tzinfo is None:
                    self._sections.append(
                        item.replace(tzinfo=tz_beijing)
                    )
                else:
                    self._sections.append(
                        item.as
    
    @property
    def exchange(self) -> str:
        return self._exchange
    
    @property
    def product(self) -> str:
        return self._product
    
    @property
    def symbol(self) -> str:
        return f'{self._exchange}.{self._product}'
    
    @property
    def count(self) -> int:
        return self._count
    
    @property
    def optional(self) -> int:
        return self._optional
    
    @property
    def sections(self) -> int:
        return self._sections
    
    def section_at(self, n: int) -> tuple:
        if n < 1 or n > self._count:
            raise ValueError('<n> should in range [1, count], got {n} instead.')
        return self._sections[(n-1)*2], self._sections[(n-1)*2+1]
    
    def __str__(self) -> str:
        return f'<ProductTradingTime(' \
               f'Symbol={self.symbol}, ' \
               f'Count={self.count}, ' \
               f'Optional={self.optional}, ' \
               f'Sections={self.sections}, ' \
               f')>'


test: ProductTradingTime = ProductTradingTime(
    exchange='SHFE',
    product='al',
    count=4,
    optional=1,
    sections=[
        dt.time(hour=21, minute=0),
        dt.time(hour=1, minute=0),
        dt.time(hour=9, minute=0),
        dt.time(hour=10, minute=15),
        dt.time(hour=10, minute=30),
        dt.time(hour=11, minute=30),
        dt.time(hour=13, minute=30),
        dt.time(hour=15, minute=0),
    ]
)

print(f'test.exchange: {test.exchange}')
print(f'test.product: {test.product}')
print(f'test.count: {test.count}')
print(f'test.optional: {test.optional}')
print(f'test.sections: {test.sections}')
print(f'test.section[2]: {test.section_at(2)}')

test.exchange: SHFE
test.product: al
test.count: 4
test.optional: 1
test.sections: [datetime.time(21, 0), datetime.time(1, 0), datetime.time(9, 0), datetime.time(10, 15), datetime.time(10, 30), datetime.time(11, 30), datetime.time(13, 30), datetime.time(15, 0)]
test.section[2]: (datetime.time(9, 0), datetime.time(10, 15))


加载所有品种的交易时间：

In [6]:
from typing import Dict
from pathlib import Path
import csv

def read_product_trading_time(csv_path: Path) -> Dict[str, ProductTradingTime]:
    if not csv_path.exists():
        raise
    result: TradingTime = {}
    with open(csv_path, mode='r', newline='', encoding='utf-8') as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            exchange = row['exchange'].upper()
            product = row['product']
            symbol = f'{exchange}.{product}'
            result[symbol] = ProductTradingTime(
                exchange=exchange,
                product=product,
                count=int(row['count']),
                optional=int(row['optional']),
                sections=[dt.time.fromisoformat(item) for item in row['section'].split(';')]
            )
    return result

from src.utility import PACKAGE_PATH

product_trading_time: Dict[str, ProductTradingTime] = read_product_trading_time(PACKAGE_PATH.joinpath('data', 'trading_time.csv'))

print(product_trading_time['SHFE.rb'])

<ProductTradingTime(Symbol=SHFE.rb, Count=4, Optional=1, Sections=[datetime.time(21, 0), datetime.time(23, 0), datetime.time(9, 0), datetime.time(10, 15), datetime.time(10, 30), datetime.time(11, 30), datetime.time(13, 30), datetime.time(15, 0)], )>
